In [1]:
import pandas as pd
df = pd.concat(objs=[
    pd.read_csv(filepath_or_buffer='/kaggle/input/gustavosta-nlp-research-prompts/train.csv', ),
    pd.read_csv(filepath_or_buffer='/kaggle/input/gustavosta-nlp-research-prompts/test.csv', )
]).sample(frac=1, random_state=2022).reset_index().drop(columns=['index']).drop_duplicates(ignore_index=True)
df.head()

,Prompt
0,tarot pele hawaiian fire goddess holding molte...
1,Lively sunny landscape of a kerala village rea...
2,portrait of a young spiderman handsome but joy...
3,"Saladin in the style of Berserk, artstation, c..."
4,"donald trump, lord of the undead, portrait pai..."


In [2]:
df.shape

(67101, 1)

In [3]:
df.nunique()

Prompt    67101
dtype: int64